In [1]:
#Bibliotecas
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from scipy.stats import anderson
import statsmodels.api as sm

Foram importadas algumas bliotecas, da linguagem de programação python, com o objetivo de facilitar a analise e manipulamento de dados.

o objetivo das importações de códigos externos foram:
pandas: Manipulação de dados em estruturas como DataFrames, ideal para carregar, limpar e transformar dados.
pyarrow.parquet: Leitura e escrita de arquivos no formato Parquet, eficiente para grandes volumes de dados.
seaborn: Criação de gráficos estatísticos atraentes e informativos, com integração facilitada ao pandas.
matplotlib.pyplot: Biblioteca básica para visualização de dados com gráficos personalizados.
numpy: Operações matemáticas e manipulação eficiente de arrays.
scipy.stats: Funções estatísticas e testes de hipóteses.
statsmodels: Modelagem estatística, regressões e testes avançados.
anderson: Teste estatístico de Anderson-Darling, útil para verificar a normalidade dos dados.

In [ ]:
#Carregando a base
df = pd.read_parquet('base_limpa.parquet')

df.head(5)

Na analise inicial da base, foi entendido que não seria necessária nenhuma remoção de colunas nos dados encontrados na base, contudo após primeiros desenvolvimentos, constatou-se que a falta de informações em cerca de 15 anos dos 21 que se encontram na analise, implicaria em dados faltantes caso fossem usadas as médias dos campos e em poucos dados para analisar nos anos restantes. Foram removidos as colunas de escanteios mandante e visitante
faltas mandante e visitante,
chute bolas parada mandante e visitante,
defesas mandante e visitante,
impedimentos mandante e visitante,
chutes mandante e visitante, por fim
chutes fora mandante e visitante.


In [ ]:
df.columns

In [ ]:
df['time_visitante'].value_counts().sort_index()

In [ ]:
df.columns

In [ ]:
df['time_mandante'].value_counts().sort_index()

In [7]:
def replace_values(df, mapping_dict):
    """
    Substitui valores nas colunas 'time_visitante' e 'time_mandante' de um DataFrame
    a partir de um dicionário de mapeamento.

    Args:
    df (pd.DataFrame): DataFrame contendo as colunas a serem modificadas.
    mapping_dict (dict): Dicionário com os valores a serem substituídos como chaves e os novos valores como valores.

    Returns:
    pd.DataFrame: DataFrame com os valores substituídos.
    """
    columns_to_replace = ['time_visitante', 'time_mandante']
    for column in columns_to_replace:
        df[column] = df[column].replace(mapping_dict)
    return df

In [8]:
mapping_dict = {"Atlético-PR": "Athletico-PR", "FC": "",
                 "EC": "",  }


df = replace_values(df, mapping_dict)

In [ ]:
#informações sobre a base
df.info()

In [11]:
#função para retirar todos os valores Nan, Null e Infinitos do df
def remove_nan_inf(df):
    """
    Remove valores NaN, Null e infinitos de um DataFrame.

    Args:
    df (pd.DataFrame): DataFrame a ser limpo.

    Returns:
    pd.DataFrame: DataFrame sem valores NaN, Null e infinitos.
    """
    return df.replace([np.inf, -np.inf], np.nan).dropna()

df = remove_nan_inf(df)

Ficando após essa primeira tratativa, 1 campo de tipo data, 13 campos de tipo númerico (11 com números reais e 2 com números inteiros) e 6 com informações categoricas (informação em texto).

Alguns campos mesmo estando com tipagem float, contiam dados que são identificados como inteiros, para adequar os dados, foi alterado o tipo dos campos para seguir o tipo inteiro.

Foram separados os campos de tipo texto na variavel de colunas_categoricas e númericas em colunas_numericas.

In [ ]:
# Alterar colunas de float para int
cols = ['publico', 'gols_mandante', 'gols_visitante', 'gols_1_tempo_mandante', 'gols_1_tempo_visitante',
        'colocacao_mandante', 'colocacao_visitante', 'ano_campeonato']

df[cols] = df[cols].astype(pd.Int64Dtype())

df.info()

In [ ]:
# Separar em Colunas numéricas
colunas_numericas = df.select_dtypes(include=['int', 'float']).columns
colunas_numericas

In [ ]:
#Separar em Colunas categóricas
colunas_categoricas = df.select_dtypes(include=['object']).columns
colunas_categoricas

In [ ]:
#Estatísticas descritivas
df.describe().T.round()


Ficando então com os seguintes campos, possuindo na tabela seus quartis, quantidade máxima, média, máximo, minimo e desvio padrão:


Ano do Campeonato
Os dados relacionados ao campo "ano_campeonato" mostram que a média dos campeonatos considerados está em 2013, com o menor ano sendo 2003 e o maior 2024. O desvio padrão de 6.29 indica uma variação moderada nos anos analisados, o que sugere que as competições cobrem um longo período de tempo. O primeiro quartil (25%) está em 2007 e o terceiro quartil (75%) em 2018, reforçando que a maioria dos dados está concentrada entre esses anos.

Rodada
O campo "rodada" possui uma média de aproximadamente 19, indicando que, em média, os jogos acontecem ao redor da metade do campeonato (dado que muitos campeonatos têm cerca de 38 rodadas). O valor mínimo é 1, enquanto o máximo é 38, cobrindo uma temporada completa. O desvio padrão de 10.95 mostra uma variação considerável nas rodadas dos jogos registrados.

Público
No que se refere ao "público", observa-se que a média de espectadores por jogo é de 15.949, com valores variando de 0 (jogos sem público, possivelmente devido a restrições) até um máximo de 82.044 pessoas. O desvio padrão de 13.943 sugere que há uma grande variação na quantidade de público, possivelmente em função do tamanho dos estádios ou da importância das partidas. O valor do 1º quartil (5787) indica que, em 25% das partidas, o público foi inferior a esse número, enquanto o 3º quartil (22.584) mostra que 75% das partidas tiveram menos de 22.584 espectadores.

Colocação das Equipes
A colocação das equipes (mandante e visitante) apresenta uma média em torno de 10 para ambos os times, sugerindo que, em geral, as equipes participantes estão distribuídas de forma relativamente homogênea na tabela do campeonato. O desvio padrão de cerca de 5.7 indica uma variação considerável na posição das equipes, com algumas partidas envolvendo times na liderança e outras com times de posições mais baixas.

Valor das Equipes
Em relação ao "valor_equipe_titular", tanto de mandantes quanto de visitantes, a média está em torno de 16 milhões. O desvio padrão elevado (aproximadamente 14 milhões) sugere que há uma grande disparidade entre os valores das equipes, refletindo a diferença de investimento e orçamento entre os clubes. O valor máximo de 100 milhões de reais para o time mandante e 96 milhões para o visitante reforça essa discrepância, enquanto os valores do 1º quartil indicam que 25% dos times possuem elencos avaliados em menos de 6 milhões.

Idade Média dos Jogadores Titulares
Quanto à idade média dos jogadores titulares, os times mandantes têm uma média de 27,29 anos e os visitantes 27,19 anos. O desvio padrão é baixo, com cerca de 1,6 para ambos, indicando que a maioria dos jogadores tem idades muito próximas dessa média. O valor máximo é de 33,6 anos para mandantes e 33,1 anos para visitantes, enquanto o mínimo fica em torno de 20 anos para ambos os lados, o que demonstra a presença tanto de jovens promessas quanto de jogadores mais experientes.

Gols
Os dados de "gols_mandante" e "gols_visitante" mostram uma média de 1,52 gols por partida para os mandantes e 1,02 para os visitantes, indicando que os times da casa tendem a marcar mais gols. A variação, refletida no desvio padrão (1,21 para mandantes e 1,02 para visitantes), também mostra que algumas partidas podem ter mais gols que a média. No entanto, o primeiro quartil para ambos os casos é 1 e 0 gols, sugerindo que uma quantidade significativa de jogos termina com poucos ou nenhum gol marcado.

Essas informações permitem uma análise detalhada sobre os jogos, revelando padrões e variações nas rodadas, no público, no desempenho das equipes e no valor dos elencos.

##Graficos

In [ ]:
# Correlações
corr = df.corr(numeric_only=True)
corr.style.background_gradient()

A tabela de correlação apresentada oferece insights sobre as relações entre variáveis relacionadas a campeonatos de futebol. A correlação vai de -1 (fortemente negativa) a +1 (fortemente positiva), com valores mais próximos de zero indicando pouca ou nenhuma relação entre as variáveis.

1. Ano do Campeonato
Correlação positiva moderada com idade média dos titulares (mandante: 0.35; visitante: 0.32): Isso sugere que, com o passar dos anos, a idade média dos jogadores aumentou. Pode ser resultado de mudanças no perfil dos atletas, com jogadores mais experientes se mantendo em atividade por mais tempo.
Correlação negativa com gols (mandante: -0.11; visitante: -0.05): Indica que, com o tempo, os times tendem a marcar menos gols, especialmente os mandantes. Isso pode refletir mudanças táticas, como defesas mais organizadas ou competições mais equilibradas.
2. Rodada
Correlação baixa com todas as variáveis: Isso sugere que a rodada do campeonato não influencia significativamente outras variáveis, como público, gols ou desempenho das equipes. Isso pode indicar que o comportamento das partidas se mantém relativamente constante ao longo da competição.
3. Público
Correlação positiva com o valor da equipe titular do mandante (0.19): Equipes com maior valor de mercado tendem a atrair mais público. Isso faz sentido, já que times mais valiosos frequentemente têm jogadores mais renomados e campanhas mais vitoriosas.
Correlação negativa com colocação do mandante (-0.29): Times que estão em posições piores na tabela tendem a ter públicos menores. Quando uma equipe vai mal no campeonato, a torcida tende a comparecer menos.
4. Colocação do Mandante e Visitante
Correlação negativa com valor da equipe titular do mandante (-0.39) e do visitante (-0.40): Times que estão em melhores posições na tabela tendem a ter elencos mais valiosos. Isso reflete a influência de investimentos financeiros no desempenho.
Correlação positiva com gols do visitante (0.14) e negativa com gols do mandante (-0.15): Times mandantes que estão melhor colocados tendem a sofrer menos gols, enquanto os visitantes, quando estão em boas posições, marcam mais gols. Isso reflete o equilíbrio e a competitividade nas partidas.
5. Valor da Equipe Titular (Mandante e Visitante)
Correlação positiva com público (mandante: 0.19): Como mencionado, equipes mais valiosas atraem mais torcedores.
Correlação positiva com gols do mandante (0.12): Equipes mandantes mais valiosas tendem a marcar mais gols, reforçando o impacto do valor do elenco no desempenho em campo.
6. Idade Média dos Titulares (Mandante e Visitante)
Correlação positiva com ano do campeonato (mandante: 0.35; visitante: 0.32): Como já discutido, isso indica uma tendência de envelhecimento dos jogadores ao longo dos anos.
Correlação baixa com outras variáveis: A idade média dos jogadores não parece ter uma relação forte com o número de gols ou a colocação das equipes, sugerindo que a experiência (medida pela idade) não tem impacto direto no desempenho em campo, pelo menos nessa análise.
7. Gols (Mandante e Visitante)
Correlação muito alta entre gols do 1º tempo e gols totais (mandante: 0.66; visitante: 0.64): Não é surpreendente, já que os gols marcados no 1º tempo contribuem diretamente para o total de gols na partida. Isso mostra que muitas das partidas têm sua tendência de placar definida desde o primeiro tempo.
Correlação negativa entre valor da equipe visitante e gols do mandante (-0.13): Times visitantes mais valiosos tendem a permitir menos gols, sugerindo que elencos mais fortes podem ser mais defensivamente eficientes.
Conclusões Gerais
A tabela revela que o valor da equipe titular está fortemente relacionado com o desempenho (gols e posição na tabela), evidenciando o papel do investimento financeiro no sucesso de uma equipe.
A idade média dos jogadores aumentou ao longo dos anos, mas esse fator tem pouco impacto sobre o número de gols ou a posição das equipes.
O público está correlacionado positivamente com o valor das equipes e negativamente com a posição na tabela, o que reforça que torcedores apoiam mais os times quando estão bem e quando têm elencos de maior valor.

In [ ]:
 # Filtrar os dados da coluna, removendo valores NaN
data = df[colunas_numericas].dropna()
df.head()

In [ ]:
# Remover NaN
df.dropna(axis = 0, inplace = True)
df.isna().sum()

###Histogramas


In [ ]:
plt.hist(df['gols_mandante'])

In [ ]:
plt.hist(df['gols_visitante'])

In [ ]:
plt.hist(df['gols_1_tempo_mandante'])

In [ ]:
plt.hist(df['gols_1_tempo_visitante'])

In [ ]:
plt.hist(df['colocacao_mandante'])

In [ ]:
plt.hist(df['colocacao_visitante'])

In [ ]:
plt.hist(df['publico'])

In [ ]:
plt.hist(df['valor_equipe_titular_mandante'])

In [ ]:
plt.hist(df['valor_equipe_titular_visitante'])

##Metodo anderson

In [ ]:
result_valor_equipe_mandante = anderson(df['valor_equipe_titular_mandante'])
result_valor_equipe_mandante

In [ ]:
result_valor_equipe_mandante.statistic

In [ ]:
result_valor_equipe_mandante.critical_values[2]

In [ ]:
qqplot(df['valor_equipe_titular_mandante'], line= 's')

In [ ]:
result_valor_equipe_visitante = anderson(df['valor_equipe_titular_visitante'])
result_valor_equipe_visitante

In [ ]:
result_valor_equipe_visitante.statistic

In [ ]:
result_valor_equipe_mandante.critical_values[2]

In [ ]:
qqplot(df['valor_equipe_titular_visitante'], line= 's')

In [ ]:
result_idade_media_titular_mandante = anderson(df['idade_media_titular_mandante'])
result_idade_media_titular_mandante

In [ ]:
result_idade_media_titular_mandante.statistic

In [ ]:
result_idade_media_titular_mandante.critical_values[2]

In [ ]:
qqplot(df['idade_media_titular_mandante'], line= 's')

In [ ]:
result_idade_media_titular_visitante = anderson(df['idade_media_titular_visitante'])
result_idade_media_titular_visitante

In [ ]:
result_idade_media_titular_visitante.statistic

In [ ]:
result_idade_media_titular_visitante.critical_values[2]

In [ ]:
qqplot(df['idade_media_titular_visitante'], line= 's')

###Fazer agrupamentos naturais

In [ ]:
def definir_regiao(time):
    # Dicionário para mapear os times às suas regiões
    regiao_times = {
        'Grêmio': 'Sul',
        'Internacional': 'Sul',
        'Palmeiras': 'Sudeste',
        'Santos': 'Sudeste',
        'São Paulo': 'Sudeste',
        'Corinthians': 'Sudeste',
        'Flamengo': 'Sudeste',
        'Fluminense': 'Sudeste',
        'Vasco da Gama': 'Sudeste',
        'Atlético-MG': 'Sudeste',
        'Botafogo': 'Sudeste',
        'Atlético-PR': 'Sul',
        'Bahia': 'Nordeste',
        'Sport Recife': 'Nordeste',
        'Vitória': 'Nordeste',
        'Coritiba': 'Sul',
        'Coritiba FC': 'Sul',
        'Goiás': 'Centro-Oeste',
        'Figueirense FC': 'Sul',
        'Cruzeiro': 'Sudeste',
        'Chapecoense': 'Sul',
        'Avaí': 'Sul',
        'Ponte Preta': 'Sudeste',
        'CSA': 'Nordeste',
        'Ponte Preta': 'Norte',
        'América-MG': 'Sudeste',
        'América-RN': 'Nordeste',
        'RB Bragantino': 'Sudeste',
        'Cuiabá-MT': 'Centro-Oeste',
        'Juventude': 'Sul',
        'Paraná': 'Sul',
        'Ceará': 'Nordeste',
        'Atlético-GO': 'Centro-Oeste',
        'Atlético-PR': 'Sul',
        'Fortaleza': 'Nordeste',
        'Criciúma': 'Sul',
        'Brasiliense': 'Centro-Oeste',
        'São Caetano': 'Sudeste',
        'Santo André': 'Sudeste',
        'Guarani': 'Sudeste',
        'Náutico': 'Nordeste',
        'Santa Cruz': 'Nordeste',
        'Portuguesa': 'Sudeste',
        'Ipatinga': 'Sudeste',
        'Goias EC':'Centro-Oeste'
        'EC Vitoria': 'Nordeste',
        'EC Bahia': 'Nordeste',





    }

In [ ]:
times_sul=[]

In [ ]:
times_norternorte=[]

In [ ]:
times_sudeste=[]

In [ ]:
times_centrooeste=[]

In [ ]:
times_noroeste=[]

In [ ]:
sns.heatmap(df.corr(numeric_only=True), annot=True)